---
self-contained: true
title: "GSB 544-Final-Regression"
author: "Ruojia Kuang"
format:
  html: 
    theme: cosmo
---

In [129]:
import pandas as pd
import numpy as np
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet 
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

In [17]:
train_data = pd.read_csv('/Users/ruojiakuang/Desktop/GSB S544 Computing and Machine Learning for Business Analytics/FInal Code/gsb-544-fall-2023-house-prices/train_new.csv')
train_data.head()

,SalePrice,PID,Lot Frontage,Lot Area,Street,Neighborhood,Bldg Type,House Style,Overall Qual,Overall Cond,...,Full Bath,Half Bath,Bedroom AbvGr,TotRms AbvGrd,Gr Liv Area,Functional,Screen Porch,Pool Area,Yr Sold,Sale Type
0,159000,531363010,80.0,9605,Pave,SawyerW,1Fam,1Story,7,6,...,1,1,3,6,1218,Typ,0,0,2009,WD
1,271900,906203120,90.0,14684,Pave,SawyerW,1Fam,1Story,7,7,...,2,0,3,7,2196,Typ,0,0,2009,WD
2,137500,916176030,NaN,14375,Pave,Timber,1Fam,SLvl,6,6,...,1,0,3,7,1344,Typ,233,0,2009,COD
3,248500,528180130,48.0,6472,Pave,NridgHt,TwnhsE,1Story,9,5,...,2,0,2,6,1456,Typ,0,0,2009,WD
4,167000,528290030,61.0,9734,Pave,Gilbert,1Fam,SLvl,7,5,...,2,1,3,7,1374,Typ,0,0,2009,WD


In [18]:
train_data.dtypes

SalePrice          int64
PID                int64
Lot Frontage     float64
Lot Area           int64
Street            object
Neighborhood      object
Bldg Type         object
House Style       object
Overall Qual       int64
Overall Cond       int64
Year Built         int64
Roof Style        object
Heating           object
Central Air       object
Electrical        object
Full Bath          int64
Half Bath          int64
Bedroom AbvGr      int64
TotRms AbvGrd      int64
Gr Liv Area        int64
Functional        object
Screen Porch       int64
Pool Area          int64
Yr Sold            int64
Sale Type         object
dtype: object

In [47]:
# Clean the dataset
train_data_cleaned = train_data.drop('PID', axis=1, errors='ignore')
train_data_cleaned = train_data.dropna(subset=['Lot Frontage'])
train_data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1835 entries, 0 to 2196
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   SalePrice      1835 non-null   int64  
 1   PID            1835 non-null   int64  
 2   Lot Frontage   1835 non-null   float64
 3   Lot Area       1835 non-null   int64  
 4   Street         1835 non-null   object 
 5   Neighborhood   1835 non-null   object 
 6   Bldg Type      1835 non-null   object 
 7   House Style    1835 non-null   object 
 8   Overall Qual   1835 non-null   int64  
 9   Overall Cond   1835 non-null   int64  
 10  Year Built     1835 non-null   int64  
 11  Roof Style     1835 non-null   object 
 12  Heating        1835 non-null   object 
 13  Central Air    1835 non-null   object 
 14  Electrical     1834 non-null   object 
 15  Full Bath      1835 non-null   int64  
 16  Half Bath      1835 non-null   int64  
 17  Bedroom AbvGr  1835 non-null   int64  
 18  TotRms A

In [48]:
# Define predictors and target variables
X = train_data_cleaned[['Lot Frontage', 'Lot Area', 'Neighborhood', 'Overall Qual', 'Overall Cond', 'Year Built', 'Gr Liv Area', 'Full Bath', 'TotRms AbvGrd', 'Central Air', 'Pool Area', 'Yr Sold']]
y = np.log(train_data_cleaned['SalePrice'])

# Identifying categorical and numerical columns in the cleaned data
categorical_columns = X.select_dtypes(include=['object']).columns.tolist()
numerical_columns = X.select_dtypes(include=['number']).columns.tolist()

categorical_columns, numerical_columns

(['Neighborhood', 'Central Air'],
 ['Lot Frontage',
  'Lot Area',
  'Overall Qual',
  'Overall Cond',
  'Year Built',
  'Gr Liv Area',
  'Full Bath',
  'TotRms AbvGrd',
  'Pool Area',
  'Yr Sold'])

# Preprocessing

In [49]:
preprocessor = ColumnTransformer(
    transformers=[
        ("standardize", StandardScaler(), numerical_columns),
        ("dummify", OneHotEncoder(sparse_output=False), categorical_columns)
    ],
     remainder = "drop"
)
preprocessor

ColumnTransformer(transformers=[('standardize', StandardScaler(),
                                 ['Lot Frontage', 'Lot Area', 'Overall Qual',
                                  'Overall Cond', 'Year Built', 'Gr Liv Area',
                                  'Full Bath', 'TotRms AbvGrd', 'Pool Area',
                                  'Yr Sold']),
                                ('dummify', OneHotEncoder(sparse_output=False),
                                 ['Neighborhood', 'Central Air'])])

# Linear Regression

In [50]:
# Create a linear regression pipieline
lr_pipeline = Pipeline(
  [('preprocessor', preprocessor),
  ("linear_regression", LinearRegression())]
)

lr_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('standardize',
                                                  StandardScaler(),
                                                  ['Lot Frontage', 'Lot Area',
                                                   'Overall Qual',
                                                   'Overall Cond', 'Year Built',
                                                   'Gr Liv Area', 'Full Bath',
                                                   'TotRms AbvGrd', 'Pool Area',
                                                   'Yr Sold']),
                                                 ('dummify',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['Neighborhood',
                                                   'Central Air'])])),
                ('linear_regression', LinearRegression())])

In [54]:
# Fit the pipeline to the full dataset
lr_pipeline.fit(X, y)

# Extract coefficients from the linear regression model
model_coefficients = lr_pipeline.named_steps['linear_regression'].coef_

# Map coefficients to their corresponding feature names
feature_names = np.concatenate([numerical_columns, preprocessor.named_transformers_['dummify'].get_feature_names_out()])
coefficients = dict(zip(feature_names, model_coefficients))

# Sort coefficients by absolute value
sorted_coefficients = dict(sorted(coefficients.items(), key=lambda item: abs(item[1]), reverse=True))

# Perform cross-validation to estimate MSE
mse_scores = -cross_val_score(lr_pipeline, X, y, cv=5, scoring='neg_mean_squared_error')
mean_mse = np.mean(mse_scores)

sorted_coefficients, mean_mse

({'Neighborhood_BrDale': -0.265045589530329,
  'Neighborhood_MeadowV': -0.26115866227539797,
  'Neighborhood_Veenker': 0.22300085252384924,
  'Neighborhood_StoneBr': 0.22083192713796135,
  'Neighborhood_NridgHt': 0.20010463396087108,
  'Neighborhood_IDOTRR': -0.1720078428906969,
  'Neighborhood_NoRidge': 0.15821474377144526,
  'Overall Qual': 0.1440402390822932,
  'Gr Liv Area': 0.14245266340499058,
  'Neighborhood_Crawfor': 0.13745372034619524,
  'Neighborhood_Timber': 0.1123737951250319,
  'Neighborhood_NPkVill': -0.11206155344672182,
  'Neighborhood_OldTown': -0.1091836353792111,
  'Neighborhood_Edwards': -0.10084587452062686,
  'Year Built': 0.08659913667454691,
  'Neighborhood_ClearCr': 0.08380528071387364,
  'Neighborhood_BrkSide': -0.08338784162407145,
  'Neighborhood_Somerst': 0.07441389524153078,
  'Central Air_N': -0.06827750323385258,
  'Central Air_Y': 0.06827750323385247,
  'Neighborhood_SWISU': -0.053420286320529635,
  'Overall Cond': 0.05328735574758477,
  'Neighborhood_

# Ridge regression

In [80]:
# Creating the ridge regression model
ridge_reg = Ridge()

# Creating a pipeline that first preprocesses the data and then applies ridge regression
ridge_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', ridge_reg)])

# Parameters for GridSearchCV
param_grid = {
    'regressor__alpha': [0.001, 0.01, 0.1, 1, 2, 3, 5, 10, 100]  # Different values for lambda (alpha in Ridge)
}

# Setting up the GridSearchCV to tune hyperparameters
grid_search = GridSearchCV(ridge_pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)

# Fit the grid search to the data
grid_search.fit(X, y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('standardize',
                                                                         StandardScaler(),
                                                                         ['Lot '
                                                                          'Frontage',
                                                                          'Lot '
                                                                          'Area',
                                                                          'Overall '
                                                                          'Qual',
                                                                          'Overall '
                                                                          'Cond',
                                                                          'Year '
                                                                          'Built',
                                                                          'Gr '
                                                                          'Liv '
                                                                          'Area',
                                                                          'Full '
                                                                          'Bath',
                                                                          'TotRms '
                                                                          'AbvGrd',
                                                                          'Pool '
                                                                          'Area',
                                                                          'Yr '
                                                                          'Sold']),
                                                                        ('dummify',
                                                                         OneHotEncoder(sparse_output=False),
                                                                         ['Neighborhood',
                                                                          'Central '
                                                                          'Air'])])),
                                       ('regressor', Ridge())]),
             param_grid={'regressor__alpha': [0.001, 0.01, 0.1, 1, 2, 3, 5, 10,
                                              100]},
             return_train_score=True, scoring='neg_mean_squared_error')

In [81]:
import pandas as pd

# Assuming grid_search is your GridSearchCV object

# Get the best hyperparameters
best_alpha = grid_search.best_params_['regressor__alpha']

# Create a DataFrame from the cross-validation results
results_df = pd.DataFrame({
    'lambda': grid_search.cv_results_['param_regressor__alpha'],
    'mse_scores': -grid_search.cv_results_['mean_test_score']
})

# Filter the DataFrame to include only rows where the alpha value matches the best_alpha
tmp = results_df[results_df['lambda'] == best_alpha]

# Print the resulting DataFrame
print(tmp)

  lambda  mse_scores
2    0.1    0.025572


# Lasso Regression

In [88]:
# Creating the Lasso regression model
lasso_reg = Lasso(max_iter=10000)  # Increasing max_iter for convergence

# Creating a pipeline that first preprocesses the data and then applies Lasso regression
lasso_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                 ('regressor', lasso_reg)])

# Parameters for GridSearchCV for Lasso
lasso_param_grid = {
    'regressor__alpha': [0.001, 0.01, 0.1, 1, 10, 100]  # Different values for lambda in Lasso
}

# Setting up the GridSearchCV for Lasso
lasso_grid_search = GridSearchCV(lasso_pipeline, lasso_param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)

# Fit the grid search to the data for Lasso
lasso_pipeline_fitted = lasso_grid_search.fit(X, y)

In [91]:
# Get the best hyperparameters
best_alpha = lasso_grid_search.best_params_['regressor__alpha']

# Create a DataFrame from the cross-validation results
results_df = pd.DataFrame({
    'lambda': lasso_grid_search.cv_results_['param_regressor__alpha'],
    'mse_scores': -lasso_grid_search.cv_results_['mean_test_score']
})

# Filter the DataFrame to include only rows where the alpha value matches the best_alpha
tmp = results_df[results_df['lambda'] == best_alpha]

# Print the resulting DataFrame
print(tmp)

  lambda  mse_scores
0  0.001    0.026953


#  Elastic Net

In [102]:
elastic_net_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', ElasticNet())
])
parameters = {'regressor__alpha' : [0.001, 0.005, 0.007, 0.01, 0.5, 2, 10],
              'regressor__l1_ratio': [0.001, 0.1, 0.2, 0.4, 0.5, 0.8, 1],
              'regressor__max_iter': [8000]
             }

en_grid_search = GridSearchCV(EN_pipeline, parameters, cv = 5, scoring='neg_mean_squared_error')
en_grid_fitted = gscv.fit(X, y)

In [97]:
# Best parameters
best_params = en_grid_fitted.best_params_
best_params

{'regressor__alpha': 0.001,
 'regressor__l1_ratio': 0.001,
 'regressor__max_iter': 8000}

In [109]:
# Get the best mean MSE score
best_mse = -en_grid_fitted.best_score_
best_mse

0.025595058160222707

# Prediction

In [124]:
test_data = pd.read_csv("/Users/ruojiakuang/Desktop/GSB S544 Computing and Machine Learning for Business Analytics/FInal Code/gsb-544-fall-2023-house-prices/test_new.csv")
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 605 entries, 0 to 604
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   PID            605 non-null    int64 
 1   Lot Frontage   605 non-null    int64 
 2   Lot Area       605 non-null    int64 
 3   Street         605 non-null    object
 4   Neighborhood   605 non-null    object
 5   Bldg Type      605 non-null    object
 6   House Style    605 non-null    object
 7   Overall Qual   605 non-null    int64 
 8   Overall Cond   605 non-null    int64 
 9   Year Built     605 non-null    int64 
 10  Roof Style     605 non-null    object
 11  Heating        605 non-null    object
 12  Central Air    605 non-null    object
 13  Electrical     605 non-null    object
 14  Full Bath      605 non-null    int64 
 15  Half Bath      605 non-null    int64 
 16  Bedroom AbvGr  605 non-null    int64 
 17  TotRms AbvGrd  605 non-null    int64 
 18  Gr Liv Area    605 non-null   

In [125]:
# Clean the dataset
test_data_cleaned = test_data.drop('PID', axis=1, errors='ignore')

X_test = test_data_cleaned[['Lot Frontage', 'Lot Area', 'Neighborhood', 'Overall Qual', 'Overall Cond', 'Year Built', 'Gr Liv Area', 'Full Bath', 'TotRms AbvGrd', 'Central Air', 'Pool Area', 'Yr Sold']]
y_pred = elastic_net_pipeline.predict(X_test)

In [126]:
submission = pd.DataFrame()
submission['PID'] = test_data['PID']
submission['SalePrice']=np.exp(y_pred)
submission

,PID,SalePrice
0,907135180,125659.978656
1,528181040,225782.054634
2,528175010,224664.861329
3,531379030,184066.011171
4,923275090,123940.140486
...,...,...
600,528174060,198348.000081
601,903400180,156539.477535
602,903227150,123810.414740
603,909250070,168220.673064


In [127]:
submission.to_csv('submission1.csv', index=False)